In [1]:
from create_graph import tidy_translators
import pandas as pd
import json
import re
import networkx as nx
import itertools
from collections import Counter
pd.set_option('display.max_columns', None)

In [2]:
with open("../data/roles.json", "r", encoding="utf8") as f:
    roles = json.load(f)
    roles_compact = list(set(roles.values()))

with open("../data/genres.json", "r", encoding="utf8") as f:
    genres = json.load(f)
    fiction = genres["fiction"]

In [3]:
erb_main = pd.read_parquet("../data/raw/erb_all_books.parquet")

In [4]:
df = erb_main.query('is_fiction == True and language == "est" and language_original.notna()')

In [5]:
df

,id,date_entered,isbn,creator,contributor,publisher,title,title_remainder,title_part_nr,title_part_nr_cleaned,title_varform,title_original,title_freeform,publication_date_control,publication_date,publication_date_cleaned,publication_decade,publication_place_control,publication_place,added_intermediate_political_jurisdiction,added_city,place_of_manufacture,manufacturer,corporate_unit,series_statement,edition_statement,edition_n,original_distribution_year,original_distribution_place,original_distribution_publisher,language,language_additional,language_original,language_note,is_fiction,is_posthumous,udc,topic_keyword,genre_keyword,geographic_keyword,chronological_keyword,corporate_keyword,person_keyword,page_count,illustrated,physical_size,print_run,price,typeface,bibliography_register,copyright_status,digitized,digitized_year,access_uri
23,b15702388,2001-11-23,9985823230,"Tralow, Johannes (1882-1968) [autor]","Malinen, Maila [tõlkija]; Sildre, Tiina (1959-...",Ajaraamat,Roxelane,romaan,<NA>,<NA>,,Roxelane,<NA>,2001,2001,2001,2000,er,Tallinn,<NA>,<NA>,Tallinn,Ühiselu,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,est,est,ger,<NA>,True,True,821.112.2-311.6,saksa [EMS022171],ilukirjandus (vormimärksõna) [EMS002472]; ajal...,<NA>,<NA>,<NA>,<NA>,415,False,22,<NA>,<NA>,<NA>,<NA>,<NA>,False,<NA>,<NA>
25,b12919688,1999-10-12,9985303067,"Shields, Carol (1935-2003) [autor]","Ehin, Andres (1940-2011) [tõlkija]; Seppel, Ly...",Varrak,Kivist päevaraamatud,romaan,<NA>,<NA>,,The stone diaries,The stone diaries; Auhind: Pulitzeri romaanipr...,1999,1999,1999,1990,er,Tallinn,<NA>,<NA>,Tallinn,Ühiselu,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,est,est,eng,<NA>,True,False,821.111,inglise [EMS022170],ilukirjandus (vormimärksõna) [EMS002472]; roma...,Kanada (riik) [EMS131809],<NA>,<NA>,<NA>,351,False,21,<NA>,<NA>,<NA>,<NA>,<NA>,False,<NA>,<NA>
27,b13564158,2000-02-22,<NA>,"Németh, Laszló (1901-1975) [autor]","Beladi, Miklós [järelsõna autor]; Hiedel, Edvi...",Eesti Raamat,Ängistus,romaan,<NA>,<NA>,,Iszony,<NA>,1969,1969,1969,1960,er,Tallinn,<NA>,<NA>,Tartu,H. Heidemanni nim. trükikoda,<NA>,XX sajandi raamat,<NA>,<NA>,<NA>,<NA>,<NA>,est,est,hun,<NA>,True,False,821.511.141-31,ungari [EMS175029],ilukirjandus (vormimärksõna) [EMS002472]; roma...,<NA>,<NA>,<NA>,<NA>,384,False,21,<NA>,<NA>,<NA>,<NA>,<NA>,False,<NA>,<NA>
28,b1292281x,1999-10-05,9789916080634,"Älimžanov, Änuar (1930-1993) [autor]","Parts, Ann [tõlkija]; Mašrapov, B [illustreerija]",Eesti Raamat,Kuller,romaan : keskmisele ja vanemale koolieale,<NA>,<NA>,<NA>,<NA>,<NA>,1980,1980,1980,1980,er,Tallinn,<NA>,<NA>,<NA>,<NA>,<NA>,15,<NA>,<NA>,<NA>,<NA>,<NA>,est,est,rus,<NA>,True,False,821.512.122-93-31,kasahhi [EMS151881],noorsookirjandus []; romaanid (vormimärksõna) ...,<NA>,<NA>,<NA>,<NA>,318,True,17,<NA>,<NA>,<NA>,<NA>,<NA>,True,2022,https://www.digar.ee/id/nlib-digar:657471
29,b15683746,2001-11-12,<NA>,"Goldberg, Anna [autor]","Lesnaja, Lidia [autor]; Seledets, Meta (1900-1...",Pedagoogiline Kirjandus,Helesinine vaip,muinasjutt,<NA>,<NA>,,Голубой ковер,<NA>,1941,1941,1941,1940,er,Tallinn,<NA>,<NA>,Tallinn,Punane Täht,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,est,est,rus,<NA>,True,False,821-93,tadžiki [EMS148300],lastekirjandus (vormimärksõna) [EMS009177]; mu...,<NA>,<NA>,<NA>,<NA>,30,True,21,3150,<NA>,<NA>,<NA>,<NA>,False,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222834,b14716161,2001-02-21,<NA>,"Wasilewska, Wanda (1905-1964) [autor]","Linask, Väino (1912-1981) [tõlkija]; Lepp, Esk...",Ilukirjandus ja Kunst,Lihtsalt armastus,jutustus,<NA>,<NA>,<NA>,<NA>,<NA>,1947,1947,1947,1940,er,Tallinn,<NA>,<NA>,Tallinn,Ühiselu,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,est,est,rus,<NA>,True,False,821.162.1-31,poola [EMS174989]; ilukirjandus [EMS002472],jutustused (vormimärksõna) [EMS009148],<NA>,<NA>,<NA>,<NA>,154,False,20,<NA>,<NA>,<NA>,<NA>,<NA>,False,<NA>

In [6]:
def extract_person_info(person_str, role=True):
    # Remove any titles enclosed in quotes
    person_str = re.sub(r': ".*?"', '', person_str)

    # Helper function to process individual dates
    def process_date(date_str):
        if not date_str:
            return None, False  # Return None and BC indicator as False
        date_str = date_str.strip()
        is_bc = False

        # Remove 'u. ' prefix indicating uncertainty
        date_str = re.sub(r'^u\. ?', '', date_str)

        # Check for 'e. Kr' (BC dates) and 'p. Kr' (AD dates)
        bc_match = re.search(r'e\. ?Kr\.?', date_str, re.IGNORECASE)
        ad_match = re.search(r'p\. ?Kr\.?', date_str, re.IGNORECASE)
        if bc_match:
            is_bc = True
        elif ad_match:
            is_bc = False  # Explicitly marked as AD

        # Remove 'e. Kr' or 'p. Kr' suffixes
        date_str = re.sub(r'(e\. ?Kr\.?|p\. ?Kr\.?)', '', date_str, flags=re.IGNORECASE).strip()

        # Convert date string to integer
        try:
            date_int = int(date_str)
            return date_int, is_bc
        except ValueError:
            return None, False  # Return None if the date is not a valid integer

    # Regular expression pattern to match the entire string
    # Matches: Name (birth_date - death_date) [role]
    pattern = r'^(.+?)\s*\((.*?)\)\s*(?:\[(.+?)\])?$'

    match = re.match(pattern, person_str)
    if match:
        name, date_range, role_str = match.groups()
        name = name.strip()
        role_str = role_str.strip().lower() if role_str and role else None

        # Split the date range into birth and death dates
        birth_date_str, _, death_date_str = date_range.partition('-')

        # Process birth and death dates
        birth_date, birth_is_bc = process_date(birth_date_str)
        death_date, death_is_bc = process_date(death_date_str)

        # If the death date is BC and the birth date is not explicitly AD, assume birth date is BC
        if death_is_bc and not birth_is_bc and birth_date is not None:
            birth_is_bc = True
            birth_date = -birth_date
        elif birth_is_bc and birth_date is not None:
            birth_date = -birth_date

        # If the death date is BC, make it negative
        if death_is_bc and death_date is not None:
            death_date = -death_date

        if role:
            return (name, birth_date, death_date, role_str)
        else:
            return (name, birth_date, death_date)

    # Handle cases with only the name and optional role
    # Matches: Name [role] or Name
    pattern_name_role = r'^(.+?)\s*(?:\[(.+?)\])?$'
    match = re.match(pattern_name_role, person_str)
    if match:
        name, role_str = match.groups()
        name = name.strip()
        role_str = role_str.strip().lower() if role_str and role else None

        if role:
            return (name, None, None, role_str)
        else:
            return (name, None, None)

    # Return None values if no pattern matched
    if role:
        return (None, None, None, None)
    else:
        return (None, None, None)

In [166]:
import networkx as nx
import itertools
from collections import Counter

def create_graph_refactored(erb, min_year, max_year, id_to_int=False):
    """
    Create a graph from the 'erb' DataFrame filtered by publication year range.

    Parameters:
    - erb: pd.DataFrame containing publication data.
    - min_year: Minimum publication year.
    - max_year: Maximum publication year.

    Returns:
    - G: A NetworkX graph with nodes and edges representing authors and their collaborations.
    """

    # Helper function to process genres
    def process_genres(genre_str, to_remove=["ilukirjandus", "e-raamatud"]):
        if isinstance(genre_str, str):
            genres = genre_str.split("; ")
            genres = [genre.split(" [")[0].split(" (")[0] for genre in genres]
            if to_remove:
                genres = [genre for genre in genres if genre not in to_remove]
            return genres
        return [None]

    # Helper function to generate node identifiers
    def get_node_identifier(name, birth_date, death_date):
        if birth_date is not None and death_date is not None:
            return f"{name} ({birth_date}-{death_date})"
        elif birth_date is not None:
            return f"{name} ({birth_date}-)"
        elif death_date is not None:
            return f"{name} (-{death_date})"
        return name

    # Filter the DataFrame using query for better readability
    df = erb.query(
        'publication_date_cleaned >= @min_year and publication_date_cleaned <= @max_year and creator.notna() and contributor.notna()'
    )

    G = nx.Graph()
    
    for _, row in df.iterrows():
        # Extract and process person data from the dataframe row
        creators = [extract_person_info(name) for name in row['creator'].split('; ') if name.strip()]
        contributors = [extract_person_info(name) for name in row['contributor'].split('; ') if name.strip()]
        all_contributors = creators + contributors
        authors = [person for person in all_contributors if person[3] == "autor"]
        translators = [person for person in all_contributors if person[3] == "tõlkija"]
        
        if authors and translators:
            people = authors + translators
            # Get node identifiers for authors and translators
            author_names = [get_node_identifier(p[0], p[1], p[2]) for p in authors]
            translator_names = [get_node_identifier(p[0], p[1], p[2]) for p in translators]
            pairs = itertools.product(author_names, translator_names)
            current_year = row['publication_date_cleaned']  # The year of the current publication
            work_info = (row['title'], current_year, current_year)
            language = row["language_original"]
            genres = set(process_genres(row["genre_keyword"]))
            
            # Update edges between authors and translators
            for node_u, node_v in pairs:
                if G.has_edge(node_u, node_v):
                    edge = G.edges[node_u, node_v]
                    edge['weight'] += 1
                    edge['works'].append(work_info)
                    edge['languages'].append(language)
                    edge['genres'].update(genres)
                    edge['activity_end'] = max(edge['activity_end'], current_year)
                    edge['activity_start'] = min(edge['activity_start'], current_year)
                else:
                    G.add_edge(
                        node_u, node_v,
                        weight=1,
                        works=[work_info],
                        languages=[language],
                        genres=genres,
                        activity_start=current_year,
                        activity_end=current_year
                    )
                    
            # Process nodes (authors and translators)
            for person in people:
                name, birth_date, death_date, role = person if len(person) == 4 else (*person, None)
                node_id = get_node_identifier(name, birth_date, death_date)

                current_node = G.nodes.get(node_id, {})

                # Update node attributes once
                current_node.update({
                    "label": node_id,
                    "label_short": name,
                    'date_of_birth': birth_date or 0,
                    'date_of_death': death_date or 0,
                })

                # Update role counts for "autor" and "tõlkija"
                if role == "autor":
                    current_node["autor_count"] = current_node.get("autor_count", 0) + 1
                elif role == "tõlkija":
                    current_node["tõlkija_count"] = current_node.get("tõlkija_count", 0) + 1

                # Update activity years
                current_node['activity_start'] = min(
                    current_node.get('activity_start', current_year), current_year
                )
                current_node['activity_end'] = max(
                    current_node.get('activity_end', current_year), current_year
                )

                G.add_node(node_id, **current_node)
                    
    # Remove self-loops (if any)
    G.remove_edges_from(nx.selfloop_edges(G))
    
    # Calculate and store the total_count and main_role for each node
    for node_id, attributes in G.nodes(data=True):
        # Calculate total role counts and determine the main role
        autor_count = attributes.get("autor_count", 0)
        tõlkija_count = attributes.get("tõlkija_count", 0)
        total_count = autor_count + tõlkija_count
        
        G.nodes[node_id]["total_count"] = total_count
        G.nodes[node_id]["main_role"] = "autor" if autor_count >= tõlkija_count else "tõlkija"
        
        # Set 'author_lang' based on the main role
        if G.nodes[node_id]["main_role"] == "autor":
            name = attributes.get('label_short', node_id)  # Use short label
            author_lang_series = df[df['creator'].str.contains(name, regex=False)]['language_original']
            author_lang = author_lang_series.mode()
            G.nodes[node_id]["author_lang"] = author_lang.iloc[0] if not author_lang.empty else "und"
        elif G.nodes[node_id]["main_role"] == "tõlkija":
            G.nodes[node_id]["author_lang"] = "tõlkija"
    
    # Update edge attributes
    for edge in G.edges:
        edge_data = G.edges[edge]
        language_counter = Counter(edge_data["languages"])
        edge_data["language"] = language_counter.most_common(1)[0][0]
        edge_data["genres"] = list(edge_data["genres"])  # Convert set to list for export
    
    # Optionally, replace node IDs with integers for better GEXF compatibility
    if id_to_int:
        G = nx.convert_node_labels_to_integers(G)

    print(f"Created graph with {G.number_of_nodes()} nodes and {G.number_of_edges()} edges")
    return G

In [167]:
G = create_graph_refactored(df, 1900, 1920)

Created graph with 785 nodes and 714 edges


In [183]:
G.nodes['Puškin, Aleksandr (1799-1837)']

{'label': 'Puškin, Aleksandr (1799-1837)',
 'label_short': 'Puškin, Aleksandr',
 'date_of_birth': 1799,
 'date_of_death': 1837,
 'autor_count': 5,
 'activity_start': 1905,
 'activity_end': 1920,
 'total_count': 5,
 'main_role': 'autor',
 'author_lang': 'rus'}

In [181]:
G.edges[('Puškin, Aleksandr (1799-1837)', 'Kuusik, Timotheos (1863-1940)')]

{'weight': 3,
 'works': [('Lumiwalge luigeke, Tsaar Saltan ja Würst Gwidon', 1920, 1920),
  ('Uinu-neiu ja seitse kangelast', 1913, 1913),
  ('Lumiwalge luigeke, Tsaar Saltan ja Würst Gwidon', 1912, 1912)],
 'languages': ['rus', 'rus', 'rus'],
 'genres': ['muinasjutud', 'lastekirjandus', 'värssjutud'],
 'activity_start': 1912,
 'activity_end': 1920,
 'language': 'rus'}

In [157]:
G.edges(data=True)

EdgeDataView([(0, 1, {'weight': 3, 'works': [('Odaw tüdruk', 1903, 1903), ('Hoia naisterahwa eest!', 1904, 1904), ('Waranduse-hoidja', 1904, 1904)], 'languages': ['hun', 'hun', 'hun'], 'genres': ['jutustused', 'jutud'], 'activity_start': 1903, 'activity_end': 1904, 'language': 'hun', 'main_genre': 'jutustused'}), (0, 166, {'weight': 1, 'works': [('Silm silma wastu', 1914, 1914)], 'languages': ['ger'], 'genres': ['jutud'], 'activity_start': 1914, 'activity_end': 1914, 'language': 'ger', 'main_genre': 'jutud'}), (0, 201, {'weight': 1, 'works': [('Waimukas wõllaroog', 1909, 1909)], 'languages': ['hun'], 'genres': ['jutustused', 'separaadid'], 'activity_start': 1909, 'activity_end': 1909, 'language': 'hun', 'main_genre': 'jutustused'}), (0, 235, {'weight': 1, 'works': [('Hingewõrgutaja', 1902, 1902)], 'languages': ['hun'], 'genres': ['separaadid', 'romaanid'], 'activity_start': 1902, 'activity_end': 1902, 'language': 'hun', 'main_genre': 'separaadid'}), (0, 69, {'weight': 1, 'works': [('Me

In [153]:
for node, attr in G.nodes(data=True):
    if attr["main_role"] not in ["autor", "tõlkija"]:
        print(node, attr)

In [119]:
for node, attr in G.nodes(data=True):
    for key, val in attr.items():
        if type(val) not in [str, int, float]:
            print(node, attr)
            break

In [ ]:
def nx_to_graphology(G):
    """
    Convert a NetworkX graph to a JSON format compatible with Graphology.js.

    Parameters:
    - G: NetworkX Graph object.

    Returns:
    - graph_data: A dictionary containing 'nodes' and 'edges' suitable for JSON serialization.
    """
    nodes = []
    edges = []

    # Process nodes
    for node_id, attrs in G.nodes(data=True):
        node = {
            'key': str(node_id),  # Ensure node_id is a string
            'attributes': attrs.copy()
        }
        node_attrs = node['attributes']

        # Add placeholder attributes if missing
        node_attrs.setdefault('x', 0.0)
        node_attrs.setdefault('y', 0.0)
        node_attrs.setdefault('size', 1.0)
        node_attrs.setdefault('color', '#000000')

        # Ensure all attribute values are JSON serializable
        for k, v in node_attrs.items():
            if isinstance(v, set):
                node_attrs[k] = list(v)
            elif isinstance(v, (tuple)):
                node_attrs[k] = list(v)
            elif isinstance(v, (nx.Graph, nx.DiGraph)):
                # Skip complex objects like graphs
                node_attrs[k] = None
            elif not isinstance(v, (str, int, float, bool, list, dict, type(None))):
                # Convert numpy and other types to basic Python types
                node_attrs[k] = json.loads(json.dumps(v, default=str))

        nodes.append(node)

    # Process edges
    for i, (u, v, attrs) in enumerate(G.edges(data=True)):
        edge = {
            'source': str(u),
            'target': str(v),
            'key': str(i),  # Use index as key
            'attributes': attrs.copy()
        }
        edge_attrs = edge['attributes']

        # Add placeholder attributes if missing
        edge_attrs.setdefault('size', 1.0)
        edge_attrs.setdefault('color', '#000000')

        # Ensure 'works' is a parsable list
        if 'works' in edge_attrs:
            # Convert list of tuples to list of lists
            works_list = []
            for work in edge_attrs['works']:
                if isinstance(work, tuple):
                    works_list.append(list(work))
                else:
                    works_list.append(work)
            edge_attrs['works'] = works_list

        # Ensure all attribute values are JSON serializable
        for k, v in edge_attrs.items():
            if isinstance(v, set):
                edge_attrs[k] = list(v)
            elif isinstance(v, (tuple)):
                edge_attrs[k] = list(v)
            elif isinstance(v, (nx.Graph, nx.DiGraph)):
                # Skip complex objects like graphs
                edge_attrs[k] = None
            elif not isinstance(v, (str, int, float, bool, list, dict, type(None))):
                # Convert numpy and other types to basic Python types
                edge_attrs[k] = json.loads(json.dumps(v, default=str))

        edges.append(edge)

    # Combine nodes and edges into graph data
    graph_data = {
        'nodes': nodes,
        'edges': edges
    }
    return graph_data

In [ ]:
nx_to_graphology(G)

In [156]:
nx.write_gexf(G, f"../data/gephi/translators_v2_1900_1940.gexf")

ValueError: too many values to unpack (expected 3)

In [130]:
nx.convert_node_labels_to_integers(G).nodes(data=True)

NodeDataView({0: {'autor_count': 8, 'activity_start': 1900, 'activity_end': 1914, 'total_count': 8, 'main_role': 'autor', 'author_lang': 'hun'}, 1: {'tõlkija_count': 9, 'activity_start': 1903, 'activity_end': 1918, 'total_count': 9, 'main_role': 'tõlkija', 'author_lang': 'tõlkija'}, 2: {'autor_count': 1, 'activity_start': 1913, 'activity_end': 1913, 'total_count': 1, 'main_role': 'autor', 'author_lang': 'ger'}, 3: {'tõlkija_count': 25, 'activity_start': 1906, 'activity_end': 1915, 'total_count': 25, 'main_role': 'tõlkija', 'author_lang': 'tõlkija'}, 4: {'tõlkija_count': 28, 'activity_start': 1907, 'activity_end': 1918, 'total_count': 28, 'main_role': 'tõlkija', 'author_lang': 'tõlkija'}, 5: {'autor_count': 4, 'activity_start': 1909, 'activity_end': 1914, 'total_count': 4, 'main_role': 'autor', 'author_lang': 'swe'}, 6: {'tõlkija_count': 3, 'activity_start': 1904, 'activity_end': 1912, 'total_count': 3, 'main_role': 'tõlkija', 'author_lang': 'tõlkija'}, 7: {'autor_count': 31, 'activity_

In [68]:
def create_graph(erb, min_year, max_year, keep_dates=True):

    # Filter
    df = erb.loc[(erb.publication_date_cleaned >= min_year) & (erb.publication_date_cleaned <= max_year)]
    df = df.loc[(df.creator.notna()) & (df.contributor.notna())]
    df["translators"] = df.contributor.apply(tidy_translators)

    G = nx.Graph()

    for _, row in df.iterrows():
        # Extract and process person data from the dataframe row
        creators = [extract_person_info(name.strip()) for name in row['creator'].split(';') if name.strip() != '']
        translators = [extract_person_info(person) for person in row["translators"]]
        if len(translators) > 0:
            people = creators + translators
            pairs = list(itertools.product([p[0] for p in creators], [p[0] for p in translators]))
            print(pairs)

            for pair in pairs:
                current_year = row['publication_date_cleaned']  # The year of the current publication
                
                if G.has_edge(pair[0], pair[1]):
                    G[pair[0]][pair[1]]['weight'] += 1
                    G[pair[0]][pair[1]]['works'].append((row['title'], current_year, current_year))
                    G[pair[0]][pair[1]]["languages"].append(row["language_original"])    # LANGUAGE
                    
                    # Update the activity_end for the edge
                    G[pair[0]][pair[1]]['activity_end'] = max(G[pair[0]][pair[1]].get('activity_end', current_year), current_year)
                    
                    # Ensure the activity_start is the smallest value
                    G[pair[0]][pair[1]]['activity_start'] = min(G[pair[0]][pair[1]].get('activity_start', current_year), current_year)
                    
                else:
                    G.add_edge(pair[0], pair[1], weight=1, activity_start=current_year, activity_end=current_year)
                    G[pair[0]][pair[1]]['works'] = [(row['title'], current_year, current_year)]
                    G[pair[0]][pair[1]]["languages"] = [row["language_original"]]

            # Process nodes (authors)
            for person in people:
                name, dates, role = person
                if keep_dates == True:
                    name = name + " " + str(dates)
                if G.has_node(name):
                    if role:
                        if role in roles.keys():
                            role_count_key = f"{roles[role]}_count"
                        else:
                            role_count_key = "muu_count"
                        G.nodes[name][role_count_key] = G.nodes[name].get(role_count_key, 0) + 1
                    G.nodes[name]['activity_end'] = max(G.nodes[name].get('activity_end', row['publication_date_cleaned']), row['publication_date_cleaned'])
                    G.nodes[name]['activity_start'] = min(G.nodes[name].get('activity_start', row['publication_date_cleaned']), row['publication_date_cleaned'])
                else:
                    node_attrs = {
                        'date_of_birth': int(dates[0]) if dates[0].isnumeric() else dates[0],
                        'date_of_death': int(dates[1]) if dates[1].isnumeric() else dates[1],
                        'activity_start': row['publication_date_cleaned'],
                        'activity_end': row['publication_date_cleaned']
                    }
                    if role:
                        node_attrs[f"{roles[role]}_count"] = 1
                    G.add_node(name, **node_attrs)

    # Remove self-loops (if any)
    G.remove_edges_from(list(nx.selfloop_edges(G)))

    # Calculate and store the total_count for each node, define the main role
    for name, attributes in G.nodes(data=True):
        node_roles = dict(zip(roles_compact, [attributes.get(f"{role}_count", 0) for role in roles_compact]))
        G.nodes[name]["total_count"] = sum(node_roles.values())
        G.nodes[name]["main_role"] = max(node_roles, key=node_roles.get)
        
        if G.nodes[name]["main_role"] == "autor":
            G.nodes[name]["author_lang"] = df.loc[df.creator.str.contains(name, regex=False)].language_original.mode().values[0]
        elif G.nodes[name]["main_role"] == "tõlkija":
            G.nodes[name]["author_lang"] = "tõlkija"
        else:
            G.nodes[name]["author_lang"] = "other"

        # # Limit years active to period when the author was alive
        # if only_living:
        #     if "date_of_death" in G.nodes[name] and type(G.nodes[name]["date_of_death"]) == int:
        #         G.nodes[name]["activity_end"] = min(G.nodes[name]["date_of_death"], G.nodes[name]["activity_end"])

    # Set the proportion of Estonian works in each edge
    for edge in G.edges:
        G.edges[edge]["language"] = Counter(G.edges[edge]["languages"]).most_common(1)[0][0]
        G.edges[edge].pop("languages")

    print(f"Created graph with {len(G.nodes)} nodes and {len(G.edges)} edges")
    return G

In [85]:
G.nodes()

NodeView(('Hayes, Mary-Rose', 'Liidja, Marja (1936-)', 'Durrell, Gerald (1925-1995)', 'Prii, Mariette', 'Garwood, Julie (1944-)', 'Vestel, Ruth', 'Tikkanen, Märta (1935-)', 'Vain, Eha (1954-2021)', 'Stine, R. L (1943-)', 'Muljar, Emily Gabrielle (1984-)', 'Muljar, Ramona Katarina', 'Christie, Agatha (1890-1976)', 'Orav, Piret (1969-)', 'Nyírő, Erzsébet', 'Sinimets, Ivar (1952-)', 'Bulgakov, Mihhail (1891-1940)', 'Vetemaa, Enn (1936-2017)', 'Reidla, Jana (1968-)', 'Varik, Maiga (1934-)', 'Ojamaa, Jüri (1932-2017)', 'Paasilinna, Arto (1942-2018)', 'Vahter, Tauno (1978-)', 'Pratchett, Terry (1948-2015)', 'Kaer, Kaaren (1978-)', 'Jones, Susanna', 'Liivamägi, Urve', 'Eichenbaum, Allan', 'Asimov, Isaac (1920-1992)', 'Õunpuu, Jaanus', 'Beaglehole, Helen', 'Jesmin, Riina (1958-)', 'Ekker, Ernst A (1937-1999)', 'Kudu, Reet (1949-)', 'Rice, Patricia', 'Ujap, A', 'Kaasik, Hugo-Tanel', 'Travers, P. L (1899-1996)', 'Haaslava, Peedu', 'Beishir, Norma', 'Lepp, Lauri', 'Cameron, Stella', 'Puskar, Rail

In [86]:
erb["genre_keyword"]

11        ilukirjandus (vormimärksõna) [EMS002472]; põne...
20        ilukirjandus (vormimärksõna) [EMS002472]; mäle...
28        ilukirjandus (vormimärksõna) [EMS002472]; põne...
29        ilukirjandus (vormimärksõna) [EMS002472]; loom...
50        ilukirjandus (vormimärksõna) [EMS002472]; põne...
                                ...                        
221527    ilukirjandus (vormimärksõna) [EMS002472]; nove...
221528    ilukirjandus (vormimärksõna) [EMS002472]; põne...
221533    aforismid []; tsitaadid []; vanasõnad []; anto...
221537    ilukirjandus (vormimärksõna) [EMS002472]; roma...
221539    ilukirjandus (vormimärksõna) [EMS002472]; roma...
Name: genre_keyword, Length: 31757, dtype: string

In [12]:
def create_translations_graph(erb, timerange, min_count=1, only_living=True, only_to_estonian=False, allowed_genres=None):
    """
    Create a graph based on the provided dataframe `erb` within a given `timerange`.
    Nodes represent authors, and edges represent co-authorship.
    """

    # Filter the dataframe based on timerange and existence of creator or contributor
    df = erb.loc[(erb.year >= timerange[0]) & (erb.year <= timerange[-1])]
    df = df.loc[(df.creator.notna()) & (df.translators.notna())]
    if only_to_estonian:
        df = df.loc[df.language_original != "est"]
    if allowed_genres:
        df = df.loc[df.subject_genre.notna()].copy()
        mask = df.subject_genre.str.replace("; ", ";").str.split(";").apply(lambda x: any(g in allowed_genres for g in x))
        df = df[mask]

    G = nx.Graph()

    for _, row in df.iterrows():
        # Extract and process person data from the dataframe row
        creators = [extract_person_info(name.strip()) for name in row['creator'].split(';') if name.strip() != '']
        translators = [extract_person_info(person) for person in row["translators"]]
        if len(translators) > 0:
            people = creators + translators
            pairs = list(itertools.product([p[0] for p in creators], [p[0] for p in translators]))
            #print(pairs)

            for pair in pairs:
                current_year = row['publication_date_cleaned']  # The year of the current publication
                
                if G.has_edge(pair[0], pair[1]):
                    G[pair[0]][pair[1]]['weight'] += 1
                    G[pair[0]][pair[1]]['works'].append((row['title'], current_year, current_year))
                    G[pair[0]][pair[1]]["languages"].append(row["language_original"])    # LANGUAGE
                    
                    # Update the activity_end for the edge
                    G[pair[0]][pair[1]]['activity_end'] = max(G[pair[0]][pair[1]].get('activity_end', current_year), current_year)
                    
                    # Ensure the activity_start is the smallest value
                    G[pair[0]][pair[1]]['activity_start'] = min(G[pair[0]][pair[1]].get('activity_start', current_year), current_year)
                    
                else:
                    G.add_edge(pair[0], pair[1], weight=1, activity_start=current_year, activity_end=current_year)
                    G[pair[0]][pair[1]]['works'] = [(row['title'], current_year, current_year)]
                    G[pair[0]][pair[1]]["languages"] = [row["language_original"]]

            # Process nodes (authors)
            for person in people:
                name, dates, role = person
                if G.has_node(name):
                    if role:
                        if role in roles.keys():
                            role_count_key = f"{roles[role]}_count"
                        else:
                            role_count_key = "muu_count"
                        G.nodes[name][role_count_key] = G.nodes[name].get(role_count_key, 0) + 1
                    G.nodes[name]['activity_end'] = max(G.nodes[name].get('activity_end', row['publication_date_cleaned']), row['publication_date_cleaned'])
                    G.nodes[name]['activity_start'] = min(G.nodes[name].get('activity_start', row['publication_date_cleaned']), row['publication_date_cleaned'])
                else:
                    node_attrs = {
                        'date_of_birth': int(dates[0]) if dates[0].isnumeric() else dates[0],
                        'date_of_death': int(dates[1]) if dates[1].isnumeric() else dates[1],
                        'activity_start': row['publication_date_cleaned'],
                        'activity_end': row['publication_date_cleaned']
                    }
                    if role:
                        node_attrs[f"{roles[role]}_count"] = 1
                    G.add_node(name, **node_attrs)

    # Remove self-loops (if any)
    G.remove_edges_from(list(nx.selfloop_edges(G)))

    # Calculate and store the total_count for each node, define the main role
    for name, attributes in G.nodes(data=True):
        node_roles = dict(zip(roles_compact, [attributes.get(f"{role}_count", 0) for role in roles_compact]))
        G.nodes[name]["total_count"] = sum(node_roles.values())
        G.nodes[name]["main_role"] = max(node_roles, key=node_roles.get)
        
        if G.nodes[name]["main_role"] == "autor":
            G.nodes[name]["author_lang"] = df.loc[df.creator.str.contains(name, regex=False)].language_original.mode().values[0]
        elif G.nodes[name]["main_role"] == "tõlkija":
            G.nodes[name]["author_lang"] = "tõlkija"
        else:
            G.nodes[name]["author_lang"] = "other"

        # Limit years active to period when the author was alive
        if only_living:
            if "date_of_death" in G.nodes[name] and type(G.nodes[name]["date_of_death"]) == int:
                G.nodes[name]["activity_end"] = min(G.nodes[name]["date_of_death"], G.nodes[name]["activity_end"])

    # Set the proportion of Estonian works in each edge
    for edge in G.edges:
        G.edges[edge]["language"] = Counter(G.edges[edge]["languages"]).most_common(1)[0][0]
        G.edges[edge].pop("languages")

    # Remove nodes below the specified min_count
    if min_count > 1:
        nodes_to_remove = [node for node, attributes in G.nodes(data=True) if attributes['total_count'] < min_count]
        G.remove_nodes_from(nodes_to_remove)

    print(f"Created graph with {len(G.nodes)} nodes and {len(G.edges)} edges")
    return G